<a href="https://colab.research.google.com/github/kashishthakur26/Data-Science/blob/main/Question_Answer_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=75c37d656ad67b03b63ae6320cc3f924190029a7d5fad88f8d3ea350d698863e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer ,util
import os
import json
import gzip

In [3]:
util.http_get('https://public.ukp.informatik.tu-darmstadt.de/reimers/sentence-transformers/datasets/simplewiki-2020-11-01.jsonl.gz', 'simplewiki-2020-11-01.jsonl.gz')

  0%|          | 0.00/50.2M [00:00<?, ?B/s]

In [4]:
model = SentenceTransformer('nq-distilbert-base-v1')

In [5]:
passages = []
with gzip.open('simplewiki-2020-11-01.jsonl.gz', 'rt', encoding='utf8') as file:
  for line in file:
    data = json.loads(line.strip())
    # print(data)
    for paragraph in data['paragraphs']:
      passages.append([data['title'], paragraph])

In [6]:
len(passages)

509663

In [7]:
from random import shuffle
shuffle(passages)

In [8]:
passages = passages[0:100_000]

In [9]:
corpus_embeddings = model.encode(passages, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

In [10]:
def get_answer(query):

  question_embedding = model.encode(query, convert_to_tensor=True)
  hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=3)[0]

  print("Results:")
  for hit in hits:
    print(passages[hit['corpus_id']])

In [11]:
get_answer("what is capital of India")
print()
get_answer("when did the cold war end?")

Results:
['Mumbai', "Mumbai (previously known as Bombay until 1996) is a natural harbor on the west coast of India, and is the capital city of Maharashtra state. It is India's largest city, and one of the world's most populous cities. It is the financial capital of India. The city is the second most-populous in the world. It has approximately 13 million people. Along with the neighboring cities of Navi Mumbai and Thane, it forms the world's 4th largest urban agglomeration. They have around 19.1 million people."]
['Panaji', 'Panaji, also known as "Panjim", is the capital city of the Indian state of Goa. With 114,405 people, it is the largest city in the state.']
['Tripura', 'Tripura () () is a state in Northeast India. It covers an area of .It is the 3rd smallest state of India. It is bigger than Lebanon but smaller than Kosovo. The capital city is Agartala.']

Results:
['Ronald Reagan', 'When Reagan visited Moscow for the fourth summit in 1988, he was seen as a celebrity by the Soviets